In [0]:
pip install lightgbm

In [0]:
pip install hyperopt

In [0]:
%pip install databricks-feature_engineering

In [0]:
%restart_python

In [0]:
import pandas as pd
import lightgbm as lgb
import pickle
import joblib
import os
from datetime import datetime, date 
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.lightgbm
from sklearn.metrics import classification_report,roc_auc_score,f1_score
import numpy as np
from mlflow.models.signature import infer_signature
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
import numpy as np
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
from databricks.feature_engineering import FeatureEngineeringClient
from databricks.feature_engineering import FeatureEngineeringClient, FeatureLookup
from sklearn.preprocessing import LabelEncoder
import json
from mlflow.tracking import MlflowClient
import requests
from sklearn.metrics import accuracy_score
from mlflow.tracking import MlflowClient
import matplotlib.pyplot as plt
from scipy.stats import chisquare

In [0]:
spark_df_sample = spark.read.table("ispl_databricks.model_logs.bd_sample")
df = spark_df_sample.toPandas()
df.head()

In [0]:
orignal_df = spark.read.table('ispl_databricks.model_logs.bd_final_feature_stores')
orignal_df = orignal_df.toPandas()
orignal_df.head()

In [0]:
model_name = 'ispl_databricks.model_logs.final_bd_model'
model_uri = f"models:/{model_name}@champion"

In [0]:
model = mlflow.pyfunc.load_model(model_uri)

In [0]:
df = df.iloc[:1000]

In [0]:
orignal_predict = model.predict(orignal_df.drop(['loan_id'], axis=1))
sample_predict = model.predict(df.drop(['loan_id'], axis=1))

In [0]:
orignal_prob = orignal_predict[:,0]
sample_prob = sample_predict[:,0]

In [0]:
plt.figure()
plt.hist(orignal_prob, bins=30, density=True)
plt.title("prignal Prediction Probability Distribution")
plt.xlabel("Probability")
plt.ylabel("Density")
plt.show()


# ---------- Production Distribution ----------
plt.figure()
plt.hist(sample_prob, bins=30, density=True)
plt.title("sample Prediction Probability Distribution")
plt.xlabel("Probability")
plt.ylabel("Density")
plt.show()

In [0]:
from scipy.stats import ks_2samp

In [0]:
ks_stat, p_value = ks_2samp(orignal_prob, sample_prob)

In [0]:
p_value

In [0]:
n_bins = 10
bins = np.linspace(0, 1, n_bins + 1)

In [0]:
ref_counts, _ = np.histogram(orignal_prob, bins=bins)

In [0]:
ref_counts

In [0]:
cur_counts, _ = np.histogram(sample_prob, bins=bins)

In [0]:
cur_counts

In [0]:
ref_freq = ref_counts / ref_counts.sum()
cur_freq = cur_counts / cur_counts.sum()

In [0]:
chi_stat, p_value = chisquare(
    f_obs=cur_freq,
    f_exp=ref_freq
)

In [0]:
p_value

In [0]:
if p_value < 0.05:
    drift_status = 'drift'
else:
    drift_status = 'no_drift'

In [0]:
dbutils.jobs.taskValues.set("drift_flag", drift_status)